# Network Anomaly Detection - Exploratory Analysis

This notebook provides interactive exploration of the Network Anomaly Detection System.

## Contents
1. Setup and Installation
2. Dataset Loading (NSL-KDD & CICIDS)
3. Data Exploration
4. Preprocessing Pipeline
5. Model Training
6. Model Evaluation
7. Visualization
8. Real-time Detection Demo

## 1. Setup and Installation

In [ ]:
# Import required libraries
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

# Set matplotlib style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# Add project to path
project_root = os.path.dirname(os.getcwd())
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print("✅ Setup complete!")
print(f"Project root: {project_root}")

## 2. Dataset Loading

### Download Instructions:

#### NSL-KDD Dataset
1. Visit: https://www.unb.ca/cic/datasets/nsl.html
2. Download `KDDTrain+.txt` and `KDDTest+.txt`
3. Place in: `../data/raw/nsl-kdd/`

#### CICIDS 2017 Dataset
1. Visit: https://www.unb.ca/cic/datasets/ids-2017.html
2. Download CSV files (Monday-Friday)
3. Place in: `../data/raw/cicids/`

**Alternative: Use synthetic data for demo**

In [ ]:
# Import data loader
from anomaly_detection.data_processing.loader import DataLoader

data_loader = DataLoader(data_path='../data/raw')

# Try loading NSL-KDD
try:
    print("Attempting to load NSL-KDD dataset...")
    train_data, test_data = data_loader.load_nsl_kdd()
    print(f"✅ NSL-KDD loaded successfully!")
    print(f"Train shape: {train_data.shape}")
    print(f"Test shape: {test_data.shape}")
    data = train_data
    dataset_name = 'NSL-KDD'
except FileNotFoundError as e:
    print(f"⚠️  NSL-KDD not found: {e}")
    print("\nGenerating synthetic data for demonstration...")
    from sklearn.datasets import make_classification
    
    X, y = make_classification(
        n_samples=5000,
        n_features=30,
        n_informative=20,
        n_redundant=5,
        n_classes=2,
        weights=[0.9, 0.1],
        random_state=42
    )
    
    feature_names = [f'feature_{i}' for i in range(30)]
    data = pd.DataFrame(X, columns=feature_names)
    data['label'] = ['normal' if label == 0 else 'anomaly' for label in y]
    dataset_name = 'Synthetic'
    print(f"✅ Synthetic data generated: {data.shape}")

print(f"\nDataset: {dataset_name}")
print(f"Shape: {data.shape}")

## 3. Data Exploration

In [ ]:
# Display basic information
print("Dataset Overview:")
print("="*60)
print(f"Number of samples: {len(data):,}")
print(f"Number of features: {len(data.columns)-1}")
print(f"\nFirst few rows:")
display(data.head())

print("\nData types:")
display(data.dtypes.value_counts())

print("\nMissing values:")
missing = data.isnull().sum()
if missing.sum() > 0:
    display(missing[missing > 0])
else:
    print("No missing values found!")

In [ ]:
# Label distribution
print("Label Distribution:")
print("="*60)
label_counts = data['label'].value_counts()
print(label_counts)
print(f"\nNormal: {label_counts.get('normal', 0):,} ({label_counts.get('normal', 0)/len(data)*100:.2f}%)")
print(f"Anomaly: {label_counts.get('anomaly', label_counts.iloc[1]):,} ({label_counts.iloc[1]/len(data)*100:.2f}%)")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar plot
label_counts.plot(kind='bar', ax=axes[0], color=['green', 'red'])
axes[0].set_title('Label Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Class')
axes[0].set_ylabel('Count')
axes[0].tick_params(axis='x', rotation=0)

# Pie chart
label_counts.plot(kind='pie', ax=axes[1], autopct='%1.1f%%', colors=['green', 'red'])
axes[1].set_title('Label Proportion', fontsize=14, fontweight='bold')
axes[1].set_ylabel('')

plt.tight_layout()
plt.show()

In [ ]:
# Statistical summary of numeric features
print("Statistical Summary:")
print("="*60)
numeric_data = data.select_dtypes(include=[np.number])
display(numeric_data.describe())

In [ ]:
# Correlation heatmap (first 10 features)
print("Feature Correlations:")
numeric_cols = data.select_dtypes(include=[np.number]).columns[:10]

plt.figure(figsize=(12, 8))
correlation = data[numeric_cols].corr()
sns.heatmap(correlation, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Feature Correlation Heatmap (First 10 Features)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 4. Preprocessing Pipeline

In [ ]:
from anomaly_detection.data_processing.preprocessor import DataPreprocessor
from anomaly_detection.data_processing.feature_engineering import FeatureEngineer

print("Preprocessing data...")
print("="*60)

# Initialize preprocessor
preprocessor = DataPreprocessor(scaling_method='standard')

# Preprocess
X, y, label_mapping = preprocessor.fit_transform(data, label_column='label', binary_classification=True)

print(f"✅ Preprocessing complete!")
print(f"Feature shape: {X.shape}")
print(f"Label shape: {y.shape}")
print(f"Label mapping: {label_mapping}")
print(f"\nClass distribution: {dict(zip(*np.unique(y, return_counts=True)))}")

In [ ]:
# Split data
print("Splitting data...")
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

print(f"Train set: {X_train.shape[0]:,} samples")
print(f"Validation set: {X_val.shape[0]:,} samples")
print(f"Test set: {X_test.shape[0]:,} samples")

# Visualize splits
fig, ax = plt.subplots(figsize=(10, 6))
splits = ['Train', 'Validation', 'Test']
sizes = [len(X_train), len(X_val), len(X_test)]
colors = ['#3498db', '#e74c3c', '#2ecc71']

ax.bar(splits, sizes, color=colors, alpha=0.7, edgecolor='black')
ax.set_ylabel('Number of Samples', fontsize=12)
ax.set_title('Data Split Distribution', fontsize=14, fontweight='bold')
for i, (split, size) in enumerate(zip(splits, sizes)):
    ax.text(i, size + 50, f'{size:,}', ha='center', fontsize=11, fontweight='bold')
plt.tight_layout()
plt.show()

## 5. Model Training

In [ ]:
from anomaly_detection.models.supervised_models import SupervisedModels
from anomaly_detection.models.unsupervised_models import UnsupervisedModels
from anomaly_detection.training.evaluator import ModelEvaluator

print("Training Models...")
print("="*60)

# Initialize models
supervised_config = {
    'random_forest': {'n_estimators': 100, 'max_depth': 20, 'random_state': 42},
    'gradient_boosting': {'n_estimators': 100, 'learning_rate': 0.1, 'random_state': 42}
}

unsupervised_config = {
    'isolation_forest': {'n_estimators': 100, 'contamination': 0.1, 'random_state': 42}
}

supervised_models = SupervisedModels(supervised_config)
unsupervised_models = UnsupervisedModels(unsupervised_config)
evaluator = ModelEvaluator()

print("Models initialized successfully!")

In [ ]:
# Train Random Forest
print("\n1. Training Random Forest...")
supervised_models.train('random_forest', X_train, y_train)
print("✅ Random Forest trained!")

In [ ]:
# Train Gradient Boosting
print("\n2. Training Gradient Boosting...")
supervised_models.train('gradient_boosting', X_train, y_train)
print("✅ Gradient Boosting trained!")

In [ ]:
# Train Isolation Forest
print("\n3. Training Isolation Forest...")
unsupervised_models.train('isolation_forest', X_train)
print("✅ Isolation Forest trained!")

## 6. Model Evaluation

In [ ]:
# Evaluate Random Forest
print("Evaluating Random Forest...")
y_pred_rf = supervised_models.predict('random_forest', X_test)
y_proba_rf = supervised_models.predict_proba('random_forest', X_test)[:, 1]
metrics_rf = evaluator.evaluate_model(y_test, y_pred_rf, y_proba_rf, 'Random Forest')
evaluator.print_evaluation('Random Forest')

In [ ]:
# Evaluate Gradient Boosting
print("Evaluating Gradient Boosting...")
y_pred_gb = supervised_models.predict('gradient_boosting', X_test)
y_proba_gb = supervised_models.predict_proba('gradient_boosting', X_test)[:, 1]
metrics_gb = evaluator.evaluate_model(y_test, y_pred_gb, y_proba_gb, 'Gradient Boosting')
evaluator.print_evaluation('Gradient Boosting')

In [ ]:
# Evaluate Isolation Forest
print("Evaluating Isolation Forest...")
y_pred_if = unsupervised_models.predict('isolation_forest', X_test)
metrics_if = evaluator.evaluate_model(y_test, y_pred_if, model_name='Isolation Forest')
evaluator.print_evaluation('Isolation Forest')

In [ ]:
# Model Comparison
print("Model Comparison:")
print("="*60)
comparison_df = evaluator.compare_models()
display(comparison_df)

# Best model
best_model, best_score = evaluator.get_best_model('f1_score')
print(f"\n🏆 Best Model: {best_model}")
print(f"   F1-Score: {best_score:.4f}")

## 7. Visualization

In [ ]:
from sklearn.metrics import confusion_matrix, roc_curve, auc

# Confusion Matrix for Random Forest
cm = confusion_matrix(y_test, y_pred_rf)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Normal', 'Anomaly'],
            yticklabels=['Normal', 'Anomaly'])
plt.title('Random Forest - Confusion Matrix', fontsize=14, fontweight='bold')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.show()

In [ ]:
# ROC Curves
plt.figure(figsize=(10, 8))

# Random Forest
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_proba_rf)
roc_auc_rf = auc(fpr_rf, tpr_rf)
plt.plot(fpr_rf, tpr_rf, label=f'Random Forest (AUC = {roc_auc_rf:.3f})', linewidth=2)

# Gradient Boosting
fpr_gb, tpr_gb, _ = roc_curve(y_test, y_proba_gb)
roc_auc_gb = auc(fpr_gb, tpr_gb)
plt.plot(fpr_gb, tpr_gb, label=f'Gradient Boosting (AUC = {roc_auc_gb:.3f})', linewidth=2)

# Baseline
plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier', linewidth=2)

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curves - Model Comparison', fontsize=14, fontweight='bold')
plt.legend(loc='lower right', fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Model Performance Comparison
metrics_to_plot = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
comparison_subset = comparison_df[['Model'] + metrics_to_plot]

fig, ax = plt.subplots(figsize=(12, 6))
x = np.arange(len(comparison_subset))
width = 0.2

for i, metric in enumerate(metrics_to_plot):
    ax.bar(x + i*width, comparison_subset[metric], width, label=metric)

ax.set_xlabel('Model', fontsize=12)
ax.set_ylabel('Score', fontsize=12)
ax.set_title('Model Performance Comparison', fontsize=14, fontweight='bold')
ax.set_xticks(x + width * 1.5)
ax.set_xticklabels(comparison_subset['Model'])
ax.legend(fontsize=10)
ax.set_ylim([0, 1.1])
ax.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

In [ ]:
# Feature Importance (Random Forest)
importances = supervised_models.get_feature_importance('random_forest')
if importances is not None and preprocessor.feature_names is not None:
    feature_names = preprocessor.feature_names
    indices = np.argsort(importances)[-20:]  # Top 20
    
    plt.figure(figsize=(10, 8))
    plt.barh(range(len(indices)), importances[indices], color='steelblue')
    plt.yticks(range(len(indices)), [feature_names[i] for i in indices])
    plt.xlabel('Importance Score', fontsize=12)
    plt.title('Top 20 Feature Importances (Random Forest)', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
else:
    print("Feature importance visualization not available")

## 8. Real-time Detection Demo

In [ ]:
# Simulate real-time detection
print("Real-time Detection Simulation")
print("="*60)

# Take a sample from test set
sample_size = 10
sample_indices = np.random.choice(len(X_test), sample_size, replace=False)
X_sample = X_test[sample_indices]
y_sample = y_test[sample_indices]

# Detect anomalies
predictions = supervised_models.predict('random_forest', X_sample)
probabilities = supervised_models.predict_proba('random_forest', X_sample)[:, 1]

print(f"\nAnalyzing {sample_size} network packets...\n")

results = []
for i, (pred, prob, true) in enumerate(zip(predictions, probabilities, y_sample)):
    status = "🔴 ANOMALY" if pred == 1 else "🟢 NORMAL"
    correct = "✓" if pred == true else "✗"
    severity = "HIGH" if prob > 0.9 else "MEDIUM" if prob > 0.7 else "LOW"
    
    print(f"Packet {i+1:2d}: {status} | Score: {prob:.4f} | Severity: {severity:6s} | {correct}")
    results.append({
        'packet': i+1,
        'prediction': pred,
        'score': prob,
        'true_label': true,
        'correct': pred == true
    })

results_df = pd.DataFrame(results)
accuracy = results_df['correct'].sum() / len(results_df) * 100
anomalies_detected = results_df['prediction'].sum()

print(f"\n{'='*60}")
print(f"Detection Accuracy: {accuracy:.1f}%")
print(f"Anomalies Detected: {anomalies_detected}/{sample_size}")
print(f"{'='*60}")

In [ ]:
# Visualize detection results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Anomaly scores
colors = ['red' if p == 1 else 'green' for p in predictions]
axes[0].bar(range(1, sample_size+1), probabilities, color=colors, alpha=0.7, edgecolor='black')
axes[0].axhline(y=0.5, color='orange', linestyle='--', linewidth=2, label='Threshold')
axes[0].set_xlabel('Packet Number', fontsize=12)
axes[0].set_ylabel('Anomaly Score', fontsize=12)
axes[0].set_title('Real-time Detection Scores', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')

# Detection summary
summary = results_df['prediction'].value_counts()
labels = ['Normal', 'Anomaly']
sizes = [summary.get(0, 0), summary.get(1, 0)]
colors_pie = ['green', 'red']
axes[1].pie(sizes, labels=labels, colors=colors_pie, autopct='%1.1f%%', startangle=90)
axes[1].set_title('Detection Summary', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

## Summary & Next Steps

### What we've accomplished:
✅ Loaded and explored network traffic data
✅ Preprocessed and split the dataset
✅ Trained multiple ML models
✅ Evaluated model performance
✅ Visualized results
✅ Demonstrated real-time detection

### Next Steps:
1. Download real datasets (NSL-KDD or CICIDS)
2. Train on full dataset with all models
3. Fine-tune hyperparameters
4. Deploy the best model
5. Launch the dashboard: `python -m anomaly_detection.main --mode dashboard`

### Resources:
- NSL-KDD: https://www.unb.ca/cic/datasets/nsl.html
- CICIDS 2017: https://www.unb.ca/cic/datasets/ids-2017.html
- Project README: ../README.md
- Quick Start: ../QUICKSTART.md

In [ ]:
# Model Performance Summary (Added)
import pandas as pd
import joblib
from pathlib import Path
from anomaly_detection.data_processing.loader import DataLoader
from anomaly_detection.data_processing.preprocessor import DataPreprocessor
from anomaly_detection.training.evaluator import ModelEvaluator
from anomaly_detection.utils.config import load_config

config = load_config('configs/config.yaml')
loader = DataLoader(config=config)
pre = DataPreprocessor(config=config)

# Load NSL-KDD train/test (no validation for quick summary)
train_raw, test_raw = loader.load_nsl_kdd()
X_train_raw = train_raw.drop(columns=['label'])
y_train_raw = train_raw['label']
X_test_raw = test_raw.drop(columns=['label'])
y_test_raw = test_raw['label']

# Fit preprocessing on train, transform test
pre.fit(X_train_raw, y_train_raw)
X_train = pre.transform_features(X_train_raw)
X_test = pre.transform_features(X_test_raw)
y_train = pre.prepare_labels(y_train_raw)
y_test = pre.prepare_labels(y_test_raw)

models_dir = Path('saved_models/supervised')
model_files = {
    'random_forest': models_dir / 'random_forest.joblib',
    'gradient_boosting': models_dir / 'gradient_boosting.joblib',
    'svm': models_dir / 'svm.joblib'
}

results = []
evaluator = ModelEvaluator()

for name, mpath in model_files.items():
    if mpath.exists():
        model = joblib.load(mpath)
        metrics = evaluator.evaluate_model(model, X_test, y_test, model_type='supervised')
        results.append({
            'Model': name,
            'Accuracy': metrics['accuracy'],
            'Precision': metrics['precision'],
            'Recall': metrics['recall'],
            'F1': metrics['f1_score'],
            'ROC_AUC': metrics['roc_auc']
        })

if results:
    df = pd.DataFrame(results).sort_values('F1', ascending=False)
    display(df)
else:
    print('No saved supervised models found to summarize.')
